In [ ]:
!pip install pyts

#Import library and data

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(20.27,10)})
import tensorflow as tf

In [ ]:
from google.colab import drive

In [ ]:
#import data
drive.mount('/content/gdrive')

In [ ]:
df = pd.read_csv('gdrive/My Drive/MBKM/gladius_v3.csv')

In [ ]:
df.reset_index(inplace = True)

In [ ]:
def late(x):
  if x == 0:
    return 'UNPAID'
  elif (x >= 1) & (x<21):
    return 'PAID'
  else:
    return 'LATE PAID'

for i in range(11, 0, -1):
  df[f'month_{i}_late'] = df['billing_{}_paymentDay'.format(i)].apply(lambda x : late(x))

#modeling

In [ ]:
amount = ['billing_{}_amountTotal'.format(i) for i in range(2,12)]
channel = ['billing_{}_channel'.format(i) for i in range (2,12)]
payDay = ['billing_{}_paymentDay'.format(i) for i in range (2,12)]
late = ['month_{}_late'.format(i) for i in range (1,12)]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
col_used2 = amount + channel + payDay + late
col_used2.append('gladius.paketradius')
col_used2.append('usage')

In [ ]:
df_model = sample_df[col_used2].loc[df['month_1_late'] != 'UNPAID']

In [ ]:
#code categorical data
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()

for column in channel:
  df_model[column] = label.fit_transform(df_model[column])
channel_class = list(label.classes_)

for column in late:
  df_model[column] = label.fit_transform(df_model[column])
late_class = list(label.classes_)

df_model['gladius.paketradius'] = label.fit_transform(df_model['gladius.paketradius'])
paket_class = list(label.classes_)

In [ ]:
X = df_model.drop('month_1_late', axis=1)
Y = df_model['month_1_late']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [ ]:
from pyts.classification import TimeSeriesForest
model = TimeSeriesForest(n_estimators = 2000, random_state=43)
model.fit(x_train, y_train)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
# evaluating the model
print('Training Accuracy :', model.score(x_train, y_train))
print('Testing Accuracy :', model.score(x_test, y_test))

# confusion matrix
cm = tf.math.confusion_matrix(labels=y_test,predictions=y_pred)

plt.figure(figsize = (10,7))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

# classification report
print('\nClassification report:')
cr = classification_report(y_test, y_pred)
print(cr)

In [ ]:
model.feature_importances_

In [ ]:
import pickle

pickle.dump(model, open('model_sample_2.p','wb'))


pickle.dump(channel_class, open('channel_sample_2.p', 'wb'))
pickle.dump(late_class, open('late_sample_2.p', 'wb'))
pickle.dump(paket_class, open('paket_sample_2', 'wb'))

In [ ]:
pickle.dump(paket_class, open('paket_sample_2.p', 'wb'))

cek label encoder di setiap billing channel, misal kolom 1 indomaret kode 0, apakah indomaret di kolom 2 juga kode 0? cek jumlah channel tiap kolomnya sama atau enggak

cara cek df['billing_1_channel].unique df['billing_2_channel].unique